In [1]:
import os
print(os.getenv("DASHSCOPE_API_KEY"))

sk-dec3caaa6d6d4350963f5ceb97dce549


In [2]:
from openai import OpenAI
import os

client = OpenAI(
    # 如果没有配置环境变量，请用API Key将下行替换为：api_key="sk-xxx"
    # 新加坡和北京地域的API Key不同。获取API Key：https://help.aliyun.com/zh/model-studio/get-api-key
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    # 以下是北京地域base_url，如果使用新加坡地域的模型，需要将base_url替换为：https://dashscope-intl.aliyuncs.com/compatible-mode/v1
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)
main='甘棗之山'
object='𤠢鼠'
line='《中山經》薄山之首，曰甘棗之山。共水出焉，而西流注于河。其上多杻木，其下有草焉，葵本而杏葉，黃華而莢實，名曰蘀，可以已瞢。有獸焉，其狀如𤠢鼠而文題，其名曰㔮，食之已癭。'

completion = client.chat.completions.create(
    model="qwen-max",
    messages=[
        {
            "role": "system",
            "content": "你是《山海经》研究专家，我需要你分析山海经中指定的主要对象和待分析对象之间的关系，然后用简单的动词返回他们之间的关系，我会给你主要对象，待分析对象和原文内容，只返回关系。"
        },
        {
            "role": "user",
            "content": f"主要对象：{main}，待分析对象：{object}，原文内容：{line}", 
        },
    ],
    extra_body={"enable_search": True}
)

json_string = completion.choices[0].message.content
print(json_string)

存在


In [ ]:
from openai import OpenAI
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# 读取CSV文件
df = pd.read_csv('shanhaijing_cooccurrence_expanded.csv')

# 尝试读取已有结果，实现断点续传
try:
    df_result = pd.read_csv('shanhaijing_cooccurrence_expanded_result.csv')
    df['关系'] = df_result['关系']
    print("检测到已有结果文件，继续处理...")
except FileNotFoundError:
    if '关系' not in df.columns:
        df['关系'] = ''
    print("开始新的处理...")

# 线程锁
lock = threading.Lock()
count = 0

def process_relation(index, row):
    """处理单个关系的函数"""
    main = row['名字']
    related_people = str(row['相关人物']).split(',') if pd.notna(row['相关人物']) else []
    line = str(row['原文行'])
    
    # 如果已经处理过，跳过
    if pd.notna(row['关系']) and row['关系'] != '':
        return index, row['关系'], True
    
    relations = []
    
    for obj in related_people:
        obj = obj.strip()
        if not obj:
            continue
            
        try:
            completion = client.chat.completions.create(
                model="qwen-max",
                messages=[
                    {
                        "role": "system",
                        "content": "你是《山海经》研究专家，我需要你分析山海经中指定的主要对象和待分析对象之间的关系，然后用简单的动词返回他们之间的关系，我会给你主要对象，待分析对象和原文内容，只返回关系，没有关系则返回无关。"
                    },
                    {
                        "role": "user",
                        "content": f"主要对象：{main}，待分析对象：{obj}，原文内容：{line}",
                    },
                ],
                extra_body={"enable_search": True}
            )
            
            relation = completion.choices[0].message.content.strip()
            relations.append(relation)
            print(f"{main} - {obj} - {relation}")
            
            time.sleep(0.1)
            
        except Exception as e:
            print(f"错误: {main} - {obj} - {str(e)}")
            relations.append("错误")
    
    return index, ','.join(relations), False

# 使用线程池处理
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = {executor.submit(process_relation, index, row): index 
               for index, row in df.iterrows()}
    
    for future in as_completed(futures):
        try:
            index, relation, is_skipped = future.result()
            
            if not is_skipped:
                with lock:
                    df.at[index, '关系'] = relation
                    count += 1
                    
                    # 每30个保存一次
                    if count % 30 == 0:
                        df.to_csv('shanhaijing_cooccurrence_expanded_result.csv', 
                                 index=False, encoding='utf-8-sig')
                        print(f"已保存 {count} 条新记录")
        except Exception as e:
            print(f"处理出错: {str(e)}")

# 最后保存一次
df.to_csv('shanhaijing_cooccurrence_expanded_result.csv', index=False, encoding='utf-8-sig')
print(f"全部完成！共处理 {count} 条新记录")

In [10]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv('shanhaijing_cooccurrence_expanded_result copy.csv')

# 定义聚合函数：将列表元素用分号分隔连接
def merge_with_separator(series, spliter=','):
    # 过滤掉空值，然后用分号连接
    valid_values = [str(x) for x in series if pd.notna(x) and str(x).strip() != '']
    return spliter.join(valid_values)

# 按名字分组并聚合其他列
grouped = df.groupby('名字', as_index=False).agg({
    '名字（拼音）': 'first',  # 保留第一个拼音值（每个名字的拼音应该是相同的）
    '原文行': lambda x: merge_with_separator(x, '|'),  # 使用竖线连接
    '相关人物': merge_with_separator,
    '章节': lambda x: merge_with_separator(x, ';'),
    '相关人物（拼音）': merge_with_separator,
    '关系': merge_with_separator
})

# 调整列顺序，将"名字（拼音）"放在"名字"后面
cols = ['名字', '名字（拼音）', '原文行', '相关人物', '章节', '相关人物（拼音）', '关系']
grouped = grouped[cols]

# 保存结果
grouped.to_csv('shanhaijing_cooccurrence_expanded_result.csv', index=False, encoding='utf-8-sig')

print(f"合并完成！原始行数: {len(df)}, 合并后行数: {len(grouped)}")
print("\n前5行预览：")
print(grouped.head())

合并完成！原始行数: 4680, 合并后行数: 2128

前5行预览：
     名字              名字（拼音）  \
0     㔮              nuo（㔮）   
1     㖶              yan（㖶）   
2     㚟             chuo（㚟）   
3  㱙塗之山  xiutuzhishan（㱙塗之山）   
4    㴬水         xieshui（㴬水）   

                                                 原文行  \
0  《中山經》薄山之首，曰甘棗之山。共水出焉，而西流注于河。其上多杻木，其下有草焉，葵本而杏葉，...   
1      又東二十七里，曰堵山，神天愚居之，是多怪風雨。其上有木焉，名曰天楄，方莖而葵狀，服者不㖶。   
2  又東北三百五十里，曰綸山，其木多梓柟，多桃枝，多柤栗橘櫾，其獸多閭麈、麢、㚟。|又東北百里，...   
3  大荒之中，有山名㱙塗之山，青水窮焉。有雲雨之山，有木名曰欒。禹攻雲雨，有赤石焉生欒，黃本，赤...   
4  又西三十里，曰瞻諸之山，其陽多金，其陰多文石。㴬水出焉，而東南流注于洛；少水出其陰，而東流注...   

                                                相关人物                       章节  \
0                        甘棗之山,𤠢鼠,薄山,共水,河,杏,杻,葵,蘀,癭,瞢                      中山經   
1                                   怪風雨,天愚,堵山,天楄,服,葵                      中山經   
2  綸山,桃枝,麈,閭,麢,櫾,梓,柤,橘,栗,柟,大堯之山,豹,虎,麢,梓,松,桑,竹,夔牛,...  中山經;中山經;中山經;中山經;中山經;中山經   
3                                 雲雨之山,青水,赤石,禹,帝,欒,枝                     大荒南經   
4                                 